In [ ]:
# Chapter 1: OpenBB Basics
from openbb import obb

obb.user.preferences.output_type = 'dataframe'

data = obb.equity.price.historical(
    'AAPL', 
    start_date='2015-01-01', 
    end_date='2015-01-05',
    interval='1d', 
    provider='yfinance'
)

print(data)

In [ ]:
# Chapter 2: Pandas Basics Index
import pandas as pd

# Create a Pandas Index
idx_1 = pd.Index([0,1,2,3,4,5])
print(idx_1)

# day range Index
days = pd.date_range('2015-01-01', periods=5, freq='D')
print(days)

# second range Index
seconds = pd.date_range('2015-01-01', periods=100, freq='s')
print(seconds)

# period range Index
prng = pd.period_range('2015Q1', '2016Q4', freq='Q')
print(prng)

# MultiIndex
tuples = [
    (pd.Timestamp('2023-07-10'), 'WMT'),
    (pd.Timestamp('2023-07-10'), 'JPM'),
    (pd.Timestamp('2023-07-10'), 'TGT'),
    (pd.Timestamp('2023-07-11'), 'WMT'),
    (pd.Timestamp('2023-07-11'), 'JPM'),
    (pd.Timestamp('2023-07-11'), 'TGT'),
]

midx = pd.MultiIndex.from_tuples(
    tuples,
    name=('date', 'ticker')
)
print(midx)

In [80]:
# Pandas Series and DataFrame
import numpy as np

def rnd():
    return np.random.randn(100)

s_1 = pd.Series(rnd(), index=seconds)
s_2 = pd.Series(rnd(), index=seconds)
s_3 = pd.Series(rnd(), index=seconds)

df = pd.DataFrame({
    'a': s_1,
    'b': s_2,
    'c': s_3
})

print(df)

                            a         b         c
2015-01-01 00:00:00 -0.394023 -1.040394  0.425366
2015-01-01 00:00:01  0.294046  1.756101 -0.783500
2015-01-01 00:00:02 -1.609396 -0.164309 -1.147128
2015-01-01 00:00:03  1.293839  1.214286 -0.499855
2015-01-01 00:00:04  0.279579  0.101349 -3.430218
...                       ...       ...       ...
2015-01-01 00:01:35  0.961154  0.247337 -1.163046
2015-01-01 00:01:36 -0.186332  0.348006  0.591708
2015-01-01 00:01:37 -0.875698  0.181161  0.634306
2015-01-01 00:01:38  0.572954 -0.387021 -1.421243
2015-01-01 00:01:39  0.492202 -1.167266  0.072743

[100 rows x 3 columns]


In [83]:
df_2 = pd.DataFrame(
    {
        'close': [158.11,144.64,132.55,158.20,146.61,134.86],
        'factor_1': [0.31, 0.24, 0.67, 0.29, 0.23, 0.71],
    },
    index=midx
)
print(df_2)

                    close  factor_1
date       ticker                  
2023-07-10 WMT     158.11      0.31
           JPM     144.64      0.24
           TGT     132.55      0.67
2023-07-11 WMT     158.20      0.29
           JPM     146.61      0.23
           TGT     134.86      0.71


In [143]:
from openbb import obb
obb.user.preferences.output_type = 'dataframe'

chains = obb.derivatives.options.chains(
    symbol='SPY',
    provider='cboe'
)

print(chains)

df_3 = chains.set_index(['expiration', 'strike', 'option_type'])
print(df_3)
print(df_3.index)

      underlying_symbol  underlying_price     contract_symbol  expiration  \
0                   SPY            670.26  SPY251002C00500000  2025-10-02   
1                   SPY            670.26  SPY251002P00500000  2025-10-02   
2                   SPY            670.26  SPY251002C00505000  2025-10-02   
3                   SPY            670.26  SPY251002P00505000  2025-10-02   
4                   SPY            670.26  SPY251002C00510000  2025-10-02   
...                 ...               ...                 ...         ...   
10377               SPY            670.26  SPY280121P00990000  2028-01-21   
10378               SPY            670.26  SPY280121C00995000  2028-01-21   
10379               SPY            670.26  SPY280121P00995000  2028-01-21   
10380               SPY            670.26  SPY280121C01000000  2028-01-21   
10381               SPY            670.26  SPY280121P01000000  2028-01-21   

       dte  strike option_type  open_interest  volume  theoretical_price  .

In [119]:
# manipulating and transforming DataFrames
from openbb import obb
import pandas as pd
import numpy as np
obb.user.preferences.output_type = 'dataframe'

asset = obb.equity.price.historical(
    'AAPL',
    provider='yfinance'
)

benchmark = obb.equity.price.historical(
    'SPY',
    provider='yfinance'
)

columns = [
    'open', 'high', 'low', 'close', 'volume', 'dividends'
]


asset.columns = columns
# benchmark.columns = columns + ['capital_gains']
benchmark.columns = columns
# print(asset)
# print(benchmark)

benchmark['price_diff'] = benchmark.close.diff()
benchmark['gain'] = benchmark.price_diff > 0
benchmark['symbol'] = 'SPY'

asset['price_diff'] = asset.close.diff()
asset['gain'] = asset.price_diff > 0
asset['symbol'] = 'AAPL'

# To avoid the FutureWarning, explicitly cast the mean to int before assignment
asset_2 = asset.copy()
asset_2.at[asset_2.index[1], 'volume'] = int(asset_2.volume.mean())

pd.concat([asset, asset_2]).drop_duplicates()


,open,high,low,close,volume,dividends,price_diff,gain,symbol
date,,,,,,,,,
2024-10-03,225.139999,226.809998,223.320007,225.669998,34044200,0.0,NaN,False,AAPL
2024-10-04,227.899994,228.000000,224.130005,226.800003,37245100,0.0,1.130005,True,AAPL
2024-10-07,224.500000,225.690002,221.330002,221.690002,39505400,0.0,-5.110001,False,AAPL
2024-10-08,224.300003,225.979996,223.250000,225.770004,31855700,0.0,4.080002,True,AAPL
2024-10-09,225.229996,229.750000,224.830002,229.539993,33591100,0.0,3.769989,True,AAPL
...,...,...,...,...,...,...,...,...,...
2025-09-29,254.559998,255.000000,253.009995,254.429993,40127700,0.0,-1.030014,False,AAPL
2025-09-30,254.860001,255.919998,253.110001,254.630005,37704300,0.0,0.200012,True,AAPL
2025-10-01,255.039993,258.790009,254.929993,255.449997,48713900,0.0,0.819992,True,AAPL


In [118]:
pd.pivot_table(
    data=asset,
    values='price_diff',
    columns='gain',
    aggfunc=['sum', 'mean', 'std']
)

sum                 mean                 std          
gain             False      True      False     True      False     True 
price_diff -339.410004  370.87001 -2.951391  2.767687  3.259732  3.120703

In [121]:
concated = pd.concat([asset, benchmark])
concated.groupby('symbol').close.ohlc()

,open,high,low,close
symbol,,,,
AAPL,225.669998,259.019989,172.419998,257.130005
SPY,567.820007,669.219971,496.480011,669.219971


In [144]:
chains = obb.derivatives.options.chains(
    'AAPL', provider='cboe'
)

expirations = chains.expiration.unique()
calls = chains[
    (chains.option_type == 'call')
    & (chains.expiration == expirations[5])
]

puts = chains[
    (chains.option_type == 'put')
    & (chains.expiration == expirations[5])
]

calls_strike = calls.set_index('strike')
puts_strike = puts.set_index('strike')

joined = calls_strike.join(
    puts_strike,
    how='left',
    lsuffix='_call',
    rsuffix='_put'
)

prices = joined[['last_trade_price_call', 'last_trade_price_put']]
prices = prices.copy()
prices['straddle_price'] = prices.sum(axis=1)
print(prices)


        last_trade_price_call  last_trade_price_put  straddle_price
strike                                                             
110.0                  144.51                  0.00          144.51
120.0                  136.28                  0.00          136.28
125.0                    0.00                  0.00            0.00
130.0                    0.00                  0.00            0.00
135.0                    0.00                  0.00            0.00
140.0                  116.30                  0.06          116.36
145.0                    0.00                  0.03            0.03
150.0                  105.04                  0.03          105.07
155.0                    0.00                  0.08            0.08
160.0                    0.00                  0.05            0.05
165.0                   91.99                  0.06           92.05
170.0                    0.00                  0.09            0.09
175.0                    0.00                  0

In [145]:
(
    chains
    .groupby(
        ['expiration', 'strike', 'option_type']
    )
    .agg({
        'last_trade_price': 'max',
        'open_interest' : 'sum'
    })
)


last_trade_price  open_interest
expiration strike option_type                                 
2025-10-03 110.0  call                   147.58              2
                  put                      0.01             62
           120.0  call                   137.43              8
                  put                      0.01            134
           125.0  call                   132.47              8
...                                         ...            ...
2028-01-21 430.0  put                      0.00              0
           440.0  call                     5.25             51
                  put                      0.00              0
           450.0  call                     4.70           1615
                  put                      0.00              0

[2440 rows x 2 columns]

In [156]:
(
    chains
    .groupby(
        ['option_type']
    )
    .apply(lambda x: (x['ask'] - x['bid']).mean(),
        include_groups=False
    )
)

option_type
call    1.178361
put     0.615402
dtype: float64

In [158]:
(
    chains
    .groupby('expiration')
    .last_trade_price
    .transform(lambda x: (x - x.mean()) / x.std())
)

0       3.531998
1      -0.636984
2       3.245252
3      -0.636984
4       3.105127
          ...   
2435   -0.762661
2436   -0.650956
2437   -0.762661
2438   -0.662658
2439   -0.762661
Name: last_trade_price, Length: 2440, dtype: float64